<h1 style = 'color:cyan'> Projeto de precificação de aluguéis com predição utilizando da Regressão Linear Múltipla </h1>
<hr>

### Bibliotecas

In [ ]:
#Bibliotecas utilizadas no projeto
import pandas as pd
import seaborn as sns
import sklearn.model_selection as ms
import sklearn.linear_model as lm
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np

### Importação dos dados

In [ ]:
#Importação de arquivo csv com os dados fornecidos.
df = pd.read_csv('teste_indicium_precificacao.csv')

### Analise exploratória dos dados

In [ ]:
#Visualização das primeiras cinco linhas do DataFrame.
df.head()

In [ ]:
#Visualização das últimas cinco linhas do DataFrame.
df.tail()

In [ ]:
#Dimensação do DataFrame.
df.shape

In [ ]:
#Identificação de valores nulos por coluna.
df.isna().sum()

Como foram identificados valores nulos nas colunas df['ultima_review'] e df['reviews_por_mes'], é necessário excluir quaisquer valores nulos das linhas.

In [ ]:
#Exclusão de linhas com qualquer valor nulo.
df = df.dropna()

In [ ]:
#Identificação dos tipos de dados por coluna.
df.dtypes

In [ ]:
#Obervação de padrão nos nomes de anúncios com preços mais caros.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)
print(df[['nome', 'price']].sort_values(by='price', ascending=False).head(100))

In [ ]:
#Exclusão de colunas desnecessárias para o modelo e identificação da correlação entre as variáveis.
df_sem_colunas_categoricas = df.drop(['id', 'nome', 'host_name', 'host_id', 'latitude', 'longitude', 'ultima_review', 'bairro_group', 'bairro', 'room_type'], axis= 1)
df_sem_colunas_categoricas.corr()

Não há um número significativo entre correlações de preço e alguma das variáveis, a maior entre as correlações é de 7,82%

In [ ]:
#Tabela descritiva das variáveis.
df_sem_colunas_categoricas.describe()

Nesta tabela descritiva, é possível observar uma média considerável, porém há valores que podem estar distorcendo a realidade, pois a média do preço está em US$ 142,33, e o valor máximo encontrado é de US$ 10000, ou seja, há outliers que necessitam ser tratados.

In [ ]:
#Identificação de outliers de preço com box plot.
sns.boxplot(y='price', data= df_sem_colunas_categoricas)

Com este box plot é possível ver nitidamente a distribuição com grande quantidade de outliers.

In [ ]:
#Histograma do preço dos aluguéis.
plt.hist(df_sem_colunas_categoricas['price'], bins= 100)
plt.show()

Outra forma de identificar outliers é utilizando de histogramas, e confirmamos a suspeita anterior.

In [ ]:
#A solução para os outliers foi encontrar um quantil onde não seria mais identificados os pontos extremos.
percentile_92 = df[['price', 'minimo_noites', 'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']].quantile(0.92)

df_clean = df[(df['price'] <= percentile_92['price']) &
              (df['minimo_noites'] <= percentile_92['minimo_noites']) & 
              (df['numero_de_reviews'] <= percentile_92['numero_de_reviews']) & 
              (df['reviews_por_mes'] <= percentile_92['reviews_por_mes']) & 
              (df['calculado_host_listings_count'] <= percentile_92['calculado_host_listings_count']) & 
              (df['disponibilidade_365'] <= percentile_92['disponibilidade_365'])]

O tratamento de outliers foi escolhido a dedo por ser o primeiro percentil onde não são apresentados outliers no box plot, sendo assim é considerado como dado limpo.

In [ ]:
#Exemplo de box plot dos dados de preço sem outliers.
sns.boxplot(y= 'price', data= df_clean)
plt.show()

In [ ]:
#Histograma de todas as variáveis tentando diminuir os outliers.
df_clean.hist(figsize= (10,10), bins = 10)
plt.show()

O histograma apresentado foi para entender melhor a distribuição dos dados tratados das variáveis, e em sua maioria, os dados apresentaram possuir um comportamento "skew right", ou distribuição assimétrica com cauda longa inclinada para a direita, positivamente viesado. Considerando este comportamento, é possível observar tendências nos dados. Quanto mais barato o preço do aluguel, mais pessoas estarão na faixa de preço, além disso em sua maioria, os inquilinos não passam de 5 dias de aluguel. O número de views totais em sua maioria estão em imóveis de até 25 reviews e no máximo 2 reviews por mês. A quantidade de imóveis listadas no site não geram grande impacto, e por fim é importante observar que a maioria dos imóveis estão com menos de 100 dias de disponibilidade, podendo gerar um preço de aluguel maior por falta de dispobilidade. 

In [ ]:
#Obtenha os tipos de quarto únicos
tipos_de_quarto = df_clean['room_type'].unique()

#Para cada tipo de quarto, crie um histograma
for tipo in tipos_de_quarto:
    df_clean[df_clean['room_type'] == tipo]['price'].hist(bins=10, alpha=0.5, figsize= (5, 5))
    plt.title(f'Histograma de preços para {tipo}')
    plt.show()

#Agrupamento de preço médio por tipo de quarto.
media_preco_por_quarto = df_clean.groupby('room_type')['price'].mean()
print(media_preco_por_quarto)

No histograma de tipos de imóveis dos anúncios e seus preços, é possível observar de novo uma tendência de distribuição assimétrica com cauda longa para a direita em tipos mais baratos, ou seja, o shared room e private room. No shared room existem anúncios de 0 dólares, o que é no mínimo intrigante para uma plataforma de aluguéis, e a maioria de seus preços não ultrapassam a barreira de US$ 100/dia, e sua média exata de US$ 60,75/dia. Private room por sua vez atinge um público-alvo diferente, propensos a pagar um pouco a mais pelo conforto de ter o seu próprio quarto, porém esperava-se uma distribuição diferente, ainda assim a maioria dos dados não ultrapassam os US$ 100, com uma média de US$ 76,14. Por último está o entire home/apt, com um salto gigantesco de distribuição no histograma. Sua média está girando em torno de US$ 150,64.

In [ ]:
#Obtenha os tipos de quarto únicos
distritos_unicos = df_clean['bairro_group'].unique()

#Para cada tipo de quarto, crie um histograma
for distrito in distritos_unicos:
    df_clean[df_clean['bairro_group'] == distrito]['price'].hist(bins=10, alpha=0.5, figsize= (5, 5))
    plt.title(f'Histograma de preços para {distrito}')
    plt.show()

#Agrupamento de preço médio por distrito.
media_preco_por_distrito = df_clean.groupby('bairro_group')['price'].mean()
print(media_preco_por_distrito)

Partindo para uma análise dos distritos, na média, o Bronx é o mais penalizado no ticket médio, considerando a localização. Mesmo sendo o mais barato em média e observando a distribuição, é possível observar algum fator externo que faça com que não tenham muitos aluguéis por la, a péssima qualidade de vida é reportada na mídia do Estados Unidos por conta da violência, provavelmente há uma correlação entre os preços baixos, e baixas quantidadades de anúncios com a baixa qualidade de vida ali presente. Staten Island tem uma média maior, porém a quantidade de anúncios conseguem ser menor que o Bronx. Pesquisando um pouco observei que o custo de vida por la é elevado, tráfego intenso, são altamente impactados ambientalmente O Queens tem um ticket médio maior do que Bronx e menor do que Staten Island, porém a quantidade de anúncios é muito superior. Mesmo sendo considerado um bairro de qualidade de vida difícil, é mais aceito em relação aos outros dois anteriores. Vale ressaltar que nenhum dos três possui média de aluguel maior ou igual a 100 dólares ao dia. Brooklyn é um dos bairros mais famosos de New York, seu ticket médio ultrapassa os US$ 100, porém o que mais se destaca é a quantidade de anúncios referente a este distrito, entre ele é o bairro com mais anúncios e maiores oportunidades. Por último esta Manhattan,o distrito com maior preço médio apresentado, além de ser o único dos cinco com menor tendência caudal para direita, ou seja, mais pessoas pagando próximo a média esperada.

Em conclusão, podemos observar uma oportunidade de negócio para focar em divulgação de aluguéis em forma de diária, com preço até no máximo US$ 145 pois está perto do valor esperado em geral.

In [ ]:
#Tentativa de visualização de tendência, porém há muitas observações para observar em um scatterplot utilizando como categoria os tipos de quarto.
fig,ax = plt.subplots(3,2, figsize=(20,10))
sns.scatterplot(x= 'minimo_noites', y= 'price',data= df_clean, ax= ax[0][0], alpha = 0.1);
sns.scatterplot(x= 'numero_de_reviews', y= 'price',data= df_clean, ax= ax[0][1], alpha = 0.1);
sns.scatterplot(x= 'reviews_por_mes', y= 'price',data= df_clean, ax= ax[1][0], alpha = 0.1);
sns.scatterplot(x= 'calculado_host_listings_count', y= 'price',data= df_clean, ax= ax[1][1], alpha = 0.1);
sns.scatterplot(x= 'disponibilidade_365', y= 'price',data= df_clean, ax= ax[2][1], alpha = 0.1);
plt.show()

Por conta da grande quantidade de dados, torna-se difícil observar linhas de tendencias com gráficos de distribuição, além de alguns estarem seguindo categorias, então não se dispersam em valores no eixo x e y, apenas no eixo y.

### Regressão Linear Múltipla Scikit Learn

In [ ]:
#Tranformação das variáveis qualitativas para variáveis dummy.
bairro_group_dummy = pd.get_dummies(df_clean['bairro_group']).iloc[:, 1:]
room_type_dummy = pd.get_dummies(df_clean['room_type']).iloc[:, 1:]

In [ ]:
#Modificação nas colunas para posteriormente facilitar na identificação das variáveis independentes e dependente.
df2 = df_clean.drop(['price', 'id', 'nome', 'host_name', 'host_id', 'latitude', 'longitude', 'ultima_review', 'bairro_group', 'bairro', 'room_type'], axis= 1)
df3 = pd.concat([df2, bairro_group_dummy, room_type_dummy], axis= 1)

In [ ]:
#Identificação das variáveis independentes e dependente.
x = df3
y = df_clean['price'].copy()

x_train = df3
y_train = df_clean['price'].copy()

In [ ]:
#Divisão do grupo controle e grupo de tratamento com a divisão de 20% para o controle e 80% para o de tratamento
x_train, x_test, y_train, y_test = ms.train_test_split(x, y, test_size= 0.2, random_state= 0)

In [ ]:
#Definição do modelo
modelo = lm.LinearRegression()
modelo.fit(x_train, y_train)

In [ ]:
#Predição dos dados considerando o modelo criado.
pred_train = modelo.predict(x_train)
pred_test = modelo.predict(x_test)

In [ ]:
#R² ou coeficiente de determinação entre variáveis independentes e a variável dependente.
r_train = r2_score(y_train, pred_train)
r_test = r2_score(y_test, pred_test)

In [ ]:
#Intercepto do modelo
modelo.intercept_

In [ ]:
#β's (ou coeficientes de inclinação ou parâmetros) das variáveis dependentes.
modelo.coef_

In [ ]:
#Gráfico de regressão do modelo
plt.figure(figsize=(10, 6))

#Plotando os valores reais
plt.scatter(y_test, pred_test, color='blue', label='Valores reais')

#Plotando a linha de regressão
plt.plot(y_test, y_test, color='red', label='Linha de regressão')

plt.xlabel('Valores Reais')
plt.ylabel('Valores Previstos')
plt.title('Valores Reais vs Valores Previstos')
plt.legend()

plt.show()

### Perguntas

2. Responda também às seguintes perguntas:

a) Supondo que uma pessoa esteja pensando em investir em um apartamento para alugar na plataforma, onde seria mais indicada a compra?

b) O número mínimo de noites e a disponibilidade ao longo do ano interferem no preço?

c) Existe algum padrão no texto do nome do local para lugares de mais alto valor?

3. Explique como você faria a previsão do preço a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?

4. Supondo um apartamento com as seguintes características:

{'id': 2595,
 'nome': 'Skylit Midtown Castle',
 'host_id': 2845,
 'host_name': 'Jennifer',
 'bairro_group': 'Manhattan',
 'bairro': 'Midtown',
 'latitude': 40.75362,
 'longitude': -73.98377,
 'room_type': 'Entire home/apt',
 'price': 225,
 'minimo_noites': 1,
 'numero_de_reviews': 45,
 'ultima_review': '2019-05-21',
 'reviews_por_mes': 0.38,
 'calculado_host_listings_count': 2,
 'disponibilidade_365': 355}

Qual seria a sua sugestão de preço?

### Respostas

2. a) Se um possível cliente do site esté em dúvida onde deve comprar um imóvel que valha mais a pena o investimento, é possível afirmar que Manhattan, pelo seu preço médio sendo o destaque entre os cinco distritos e por uma melhor qualidade de vida em comparação aos outros apresentados.

b) Observando todos os coeficientes, inclusive das dummies, identifica-se que não há valores nulos, então todos os betas e suas variáveis independentes impactam na variável dependente.

c) Os anúncios mais caros sempre deixam a entender que é um imóvel de luxo e as localizações tendem a ser próximos de locais muito famosos de Nova York por filmes.

3. Para a previsão do preço foi utilizada uma regressão linear múltipla, com uma variável dependente y sendo o preço do aluguel anunciado e as variáveis independentes divididas entre duas dummies de valor númerico categórico, sendo elas o distrito e o tipo de imóvel anunciado. Eu descartei do estudo as colunas ['id', 'nome', 'host_name', 'host_id', 'latitude', 'longitude', 'ultima_review', 'bairro'] em geral por não ver sentido da correção delas estarem presente no estudo, e além disto, quando tentei fazer a dummy debairro, geraram muitas colunas, e gerando também muito coeficientes, aproximadamente 260 coeficientes. Com a ajuda do pacote Scikit Learn eu consegui criar o modelo para a predição de aluguéis com um coeficiente de determinação de 48,25%, ou seja, 48,25% dos resultados podem ser explicados com o modelo de predição. O R² foi escolhido por ser uma das primeiras medidas de eficiência de um modelo, e considerando as tentativas feitas neste projeto onde eu chega apenas a 14 e 18%, estou muito feliz com o resultado, independentemente se esteja correto.

4. De acordo com o modelo de machine learning desenvolvido, há uma chance de 48% de que a sugestão de preço para este anúncio seja US$ 135,12

In [34]:
dados = {'id': 2595,
 'nome': 'Skylit Midtown Castle',
 'host_id': 2845,
 'host_name': 'Jennifer',
 'bairro_group': 'Manhattan',
 'bairro': 'Midtown',
 'latitude': 40.75362,
 'longitude': -73.98377,
 'room_type': 'Entire home/apt',
 'price': 225,
 'minimo_noites': 1,
 'numero_de_reviews': 45,
 'ultima_review': '2019-05-21',
 'reviews_por_mes': 0.38,
 'calculado_host_listings_count': 2,
 'disponibilidade_365': 355}

# Converta para DataFrame
df = pd.DataFrame([dados])

# Defina as categorias para 'bairro_group' e 'room_type'
categorias_bairro = ['Manhattan', 'Brooklyn', 'Queens', 'Staten Island', 'Bronx']
categorias_room_type = ['Entire home/apt', 'Private room', 'Shared room']

# Converta 'bairro_group' e 'room_type' para o tipo de categoria categórica
df['bairro_group'] = pd.Categorical(df['bairro_group'], categories=categorias_bairro)
df['room_type'] = pd.Categorical(df['room_type'], categories=categorias_room_type)

# Crie variáveis dummy para 'bairro_group' e 'room_type'
dummies = pd.get_dummies(df[['bairro_group', 'room_type']])

# Selecione as outras colunas
restante_das_variaveis = df[['minimo_noites', 'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']]

# Concatene as variáveis dummy e as outras colunas
x_observado = pd.concat([dummies, restante_das_variaveis], axis=1)

# Reordene as colunas para que correspondam à ordem em que o modelo foi treinado
x_observado = x_observado.reindex(columns=x_train.columns, fill_value=0)

# Faça a previsão
previsao = modelo.predict(x_observado)

print(f'A sugestão de preço é: US${previsao[0]}')

A sugestão de preço é: US$133.62541905428367
